# Import

In [1]:
%pip install spotipy
%pip install lyricsgenius
%pip install pandas
#git clone https://github.com/JayChen35/spotify-to-mp3-python.git
%pip install yt_dlp
!pip3 install spotipy && pip3 install youtube_dl && pip3 install youtube_search && pip3 install yt_dlp && pip3 install ffprobe && pip3 install ffmpeg

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re

# Load data

## Data

In [2]:
df = pd.read_csv('data_incial_completa.csv').astype({ 'valence_tags': np.float32, 'arousal_tags':np.float32, 'dominance_tags':np.float32})

In [3]:
df


,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,cuadrante
0,https://www.last.fm/music/eminem/_/%2527till%2...,'Till I Collapse,Eminem,aggressive,6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,3
1,https://www.last.fm/music/metallica/_/st.%2banger,St. Anger,Metallica,aggressive,8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,3
2,https://www.last.fm/music/rick%2bross/_/speedi...,Speedin',Rick Ross,aggressive,1,3.080000,5.870000,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,3
3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,M.I.A.,aggressive,13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,3
4,https://www.last.fm/music/dope/_/die%2bmf%2bdie,Die MF Die,Dope,aggressive,7,3.771177,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal,3
...,...,...,...,...,...,...,...,...,...,...,...,...
61625,https://www.last.fm/music/quietdrive/_/secret,Secret,Quietdrive,transparent,1,5.370000,3.450000,5.330000,909ba2e0-6076-4ad8-8be9-a335725de4ef,2bRIsZ92JRKlvQOZlyR9CO,NaN,0
61626,https://www.last.fm/music/medications/_/the%2b...,The Last of the Rest Was the End,Medications,transparent,1,5.370000,3.450000,5.330000,ffd10ae8-858a-4b5e-819e-64f1174bbd42,7o3Np7cho9cBCrNDokxzYC,NaN,0
61627,https://www.last.fm/music/daniel%2blanois/_/lo...,Lovechild,Daniel Lanois,transparent,2,6.685000,4.405000,5.625000,16c3d394-c4d4-4dc2-bbf1-b2bef3ac861c,4fVObxldDzxxRD6a5Eth9s,indie,0
61628,https://www.last.fm/music/tapage/_/last%2binhale,Last Inhale,Tapage,transparent,6,4.301667,3.341667,4.466667,b744fdbc-7b4c-49a1-a3d6-99047d82eb34,5WxwRwUQ4R4L46VEm3213y,ambient,0


# Audios

## Oauth

In [4]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

os.environ["SPOTIPY_CLIENT_ID"]= "754cb67c8ae644dda9ed31be5eef1b21"
os.environ["SPOTIPY_CLIENT_SECRET"]= "7aa68cf2aa924657adb9c316fc8b467a"
os.environ["SPOTIPY_REDIRECT_URI"]= "http://localhost:8080/callback"

scopes = "user-follow-modify playlist-modify-private playlist-modify-public"
token = spotipy.util.prompt_for_user_token("my_app", scopes)  # "my_app" can be anything
sp = spotipy.Spotify(auth=token)

# Get your user ID
me = sp.me()
my_user_id = me['id']


## crear playlist

In [7]:
new_playlist = sp.user_playlist_create(user=my_user_id, name="Canciones", public=True, description="Tesis test")
#sp.user_playlist_create(user="padovil", name="Canciones", public=True, description="Tesis test") #ya esta creado
#spotify:playlist:7Ft3sUJWzCHZrCqLBI5a0j

## Playlist add items

In [12]:
#SOLO EJECUTAR UNA VEZ
#sp.playlist_add_items(playlist_id="spotify:playlist:0VFtEQt62WRCb6arsHkKaJ", items=df["spotify_id"])

# Create the playlist and *capture the result*

new_playlist_id = new_playlist['id']
items_to_add = df["spotify_id"].tolist()
chunk_size = 100

for i in range(0, len(items_to_add), chunk_size):
    chunk = items_to_add[i:i + chunk_size]
    try:
        sp.playlist_add_items(playlist_id=new_playlist_id, items=chunk)
        print(f"Added chunk {i // chunk_size + 1} of {len(items_to_add) // chunk_size + 1}")
    except spotipy.SpotifyException as e:
        if e.http_status == 400 and "Playlist size limit reached" in str(e):  # Check for the specific error
            print(f"Error adding chunk {i // chunk_size + 1}: {e}")
            print("Playlist size limit reached. Stopping.")
            break  # Stop adding tracks
        else:
            print(f"Error adding chunk {i // chunk_size + 1}: {e}")
            #... (your other error handling code)
    except Exception as e:
        print(f"A general error has occurred: {e}")
        break

HTTP Error for POST to https://api.spotify.com/v1/playlists/2ulTRxWx3u8tdldXiP0Zqh/tracks with Params: {'position': None} returned 400 due to Playlist size limit reached. Consider removing items from the playlist before retrying the request


Error adding chunk 1: http status: 400, code:-1 - https://api.spotify.com/v1/playlists/2ulTRxWx3u8tdldXiP0Zqh/tracks:
 Playlist size limit reached. Consider removing items from the playlist before retrying the request, reason: None
Playlist size limit reached. Stopping.


In [ ]:
playlist_tracks = sp.playlist_items(new_playlist_id)  # Get playlist tracks
added_track_ids = [track['track']['id'] for track in playlist_tracks['items']]  # Extract track IDs

added_track_ids

In [20]:
# Compare with your DataFrame (assuming 'spotify_id' column)
expected_track_ids = df['spotify_id'].tolist()[:len(added_track_ids)]  # Get expected IDs

if added_track_ids == expected_track_ids:
    print("All expected tracks were added to the playlist.")
else:
    print("Some tracks are missing or in the wrong order.")

All expected tracks were added to the playlist.


## Audio names

In [6]:
stringParams = ""
audios = []
for k, song in enumerate(df["spotify_id"]):
    
    name = "'" + str(k+1) + "-" +  str(df["track"].iloc[k]).replace(" ","-").replace("'","") + "-" + str(df["artist"].iloc[k]).replace(" ","-").replace("'","")   + "',"
    stringParams += name
    audios.append(name[1:-2])

audio = pd.DataFrame()
audio["audios"] = audios



In [7]:
stringParams

'\'1-Till-I-Collapse-Eminem\',\'2-St.-Anger-Metallica\',\'3-Speedin-Rick-Ross\',\'4-Bamboo-Banga-M.I.A.\',\'5-Die-MF-Die-Dope\',\'6-Step-Up-Drowning-Pool\',\'7-Feedback-Kanye-West\',\'8-7-Words-Deftones\',\'9-Limp-Fiona-Apple\',\'10-Sweet-Amber-Metallica\',\'11-Depression-Black-Flag\',\'12-Comprachicos-Pendulum\',\'13-When-Girls-Telephone-Boys-Deftones\',\'14-Two-Words-Kanye-West\',\'15-What-I-See-Black-Flag\',\'16-Requiem-Lamb-of-God\',\'17-Room-13-Black-Flag\',\'18-Shake-Ya-Ass-Mystikal\',\'19-Combat-Deftones\',\'20-Glue-Man-Fugazi\',\'21-Boom-Boom-The-Animals\',\'22-Boys-Wanna-Be-Her-Peaches\',\'23-No-More-Black-Flag\',\'24-Death-Trend-Setta-Crossfade\',\'25-Padded-Cell-Black-Flag\',\'26-Angel-Duster-Run-the-Jewels\',\'27-Cancer-Sick-Puppies\',\'28-Knives-and-Pens-Black-Veil-Brides\',\'29-This-Spiteful-Snake-Meshuggah\',\'30-Cuckoo-for-Caca-Faith-No-More\',\'31-The-Coming-Of-Spring-The-Rapture\',\'32-Custer-Slipknot\',\'33-Welcome-To-The-Fold-Filter\',\'34-Black-Sevendust\',\'35-Fea

## Save df

In [24]:
audio.to_csv('Data_audios/data_audios.csv', header=True, index=False)

## Download audios

In [ ]:
!python spotify-to-mp3-python/spotify_to_mp3.py "1-Till-I-Collapse-Eminem"

# Librosa

In [6]:
%pip install librosa
!pip install --upgrade numpy
!pip install --upgrade scipy
!pip install --upgrade soundfile
!pip install --upgrade librosa

Note: you may need to restart the kernel to use updated packages.


Extraer características temporales:

La función "beat_track" de librosa detecta el tempo (ritmo) del audio y devuelve el valor de tempo en BPM y los tiempos de inicio de cada beat.

Extraer características espectrales:

La función "chroma_stft" de librosa calcula un cromagrama, que es una representación del espectro de frecuencia de un audio. La función "mfcc" calcula los coeficientes cepstrales de frecuencia mel (MFCC), que son características que describen la forma en que se distribuyen las frecuencias de un audio.


Extraer características armónicas:

La función "hpss" de librosa separa un audio en dos señales: una señal armónica y otra percusiva. La señal armónica contiene las frecuencias fundamentales y los armónicos del audio, mientras que la señal percusiva contiene los componentes rítmicos y de percusión.

In [7]:
!pip install  librosa
!pip install libsndfile 
!pip install  ffmpeg 
!pip install soundfile audioread

ERROR: Could not find a version that satisfies the requirement libsndfile (from versions: none)
ERROR: No matching distribution found for libsndfile


In [8]:
import librosa
import numpy as np
import os
import subprocess  # For FFmpeg check
import pandas as pd
import re

# --- FFmpeg Installation Check ---
try:
    subprocess.run(["ffmpeg", "-version"], check=True, capture_output=True)
    print("FFmpeg is installed and accessible.")
except FileNotFoundError:
    print("FFmpeg is NOT found. Please install it using 'conda install -c conda-forge ffmpeg'")
    exit()  # Exit the script if FFmpeg is missing
except subprocess.CalledProcessError as e:
    print(f"FFmpeg error: {e.stderr.decode()}")
    exit()

# --- Audioread Backend Check ---
import audioread

FFmpeg is installed and accessible.


In [9]:
print("Available audioread backends:")
for backend in audioread.available_backends():
    print(f"- {type(backend).__name__}")

    

Available audioread backends:
- type
- type


In [10]:
# --- Paso 0: Configuración y Funciones Auxiliares ---

def extract_audio_features(file_path, song_id_txt, sr=None, hop_length=512):
    """
    Extrae características de un archivo de audio, sincronizadas con el beat.

    Args:
        file_path: Ruta al archivo de audio.
        song_id_txt: El ID de la canción (nombre del archivo .txt).
        sr: Frecuencia de muestreo (opcional).  Si es None, usa la original.
        hop_length:  Tamaño del salto (en samples) para características como MFCC.

    Returns:
        Diccionario con características sincronizadas con el beat y songId,
        o None si hay error.
    """
    try:
        y, sr = librosa.load(file_path, sr=sr)

        # Separar componentes armónicos y percusivos
        y_harmonic, y_percussive = librosa.effects.hpss(y)

        # Seguimiento del beat (en la señal percusiva)
        tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)

        # --- MFCCs ---
        mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
        mfcc_delta = librosa.feature.delta(mfcc)
        # Sincronizar con el beat (media)
        beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)

        # --- Cromagrama ---
        chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr, hop_length=hop_length)
        # Sincronizar con el beat (mediana)
        beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)

        # --- Combinar Características ---
        beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

        # Convertir a lista y crear diccionario
        features = {
            'song_id': song_id_txt.replace(".txt", ""),
            'beat_features': beat_features.T.tolist()  # Transponer para tener (beats, features)
            # Puedes añadir más características aquí si quieres,
            #  pero DEBEN estar sincronizadas con el beat.
        }
        return features

    except Exception as e:
        print(f"Error al procesar {file_path}: {e}")
        return None

In [11]:
# --- Paso 1: Cargar Datos y Directorios ---

df_lyric = pd.read_csv('Data_completa_enrriquecida.csv',  index_col=0)

audio_dir = "Canciones"
output_dir = "audio_features"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [12]:
# --- Paso 2:  Renombrar Archivos (Adaptado) ---

# Crear un diccionario de mapeo: songId (del DataFrame) -> nombre_archivo_audio
file_mapping = {}
for filename in os.listdir(audio_dir):
    if filename.endswith(".wav"):
        #  Intentar extraer artista y título del nombre del archivo de AUDIO.
        try:
            match = re.match(r"(.+?) - (.+?)\.wav", filename) #Mismo regex de antes
            if match:
                audio_artist = match.group(1).strip().lower().replace(" ", "-").replace("'", "")
                audio_track = match.group(2).strip().lower().replace(" ", "-").replace("'", "")

                # Buscar la coincidencia con el songId (que ahora es el nombre del .txt)
                for index, row in df_lyric.iterrows():
                    song_id_txt = row['songId'].lower()
                    #Comparacion con el songId
                    if song_id_txt.startswith(audio_track) and song_id_txt.replace(".txt","").endswith(audio_artist):
                        file_mapping[row['songId']] = filename  # songId -> nombre_archivo_audio
                        break #Sale del bucle si encuentra coincidencia

        except Exception as e:
            print(f"Error al procesar {filename}: {e}")
            continue


# Renombrar archivos (si es necesario)
for song_id_txt, old_audio_name in file_mapping.items():
    new_audio_name = song_id_txt.replace(".txt", ".wav")  # Nuevo nombre:  songId.wav
    old_path = os.path.join(audio_dir, old_audio_name)
    new_path = os.path.join(audio_dir, new_audio_name)

    if not os.path.exists(new_path):
        try:
            os.rename(old_path, new_path)
            print(f"Renombrado: {old_audio_name} -> {new_audio_name}")
        except Exception as e:
            print(f"Error al renombrar: {e}")
    else:
        print(f"El archivo {new_path} ya existe. No se renombró.")

In [13]:
# --- Paso 3: Extracción y Almacenamiento de Características (Sincronizadas con el Beat) ---
all_features = []
for filename in os.listdir(audio_dir):
    if filename.endswith(".wav"):
        song_id_txt = filename.replace(".wav", ".txt")  # Obtener el songId (.txt)
        file_path = os.path.join(audio_dir, filename)
        features = extract_audio_features(file_path, song_id_txt)
        if features:
            all_features.append(features)

# Crear y guardar DataFrame
df_audio_features = pd.DataFrame(all_features)

# Guardar como CSV (¡o un formato mejor, como .npy o HDF5 si tienes muchas características!)
output_file = os.path.join(output_dir, "audio_features_beat.csv") #Nombre descriptivo
df_audio_features.to_csv(output_file, index=False)
print(f"Características de audio (beat-sync) guardadas en: {output_file}")
print(df_audio_features.head())

C:\Users\andre\AppData\Local\Temp\ipykernel_7904\4019339277.py:18: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=sr)
c:\Users\andre\anaconda3\envs\pycaret_env\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
C:\Users\andre\AppData\Local\Temp\ipykernel_7904\4019339277.py:18: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=sr)
c:\Users\andre\anaconda3\envs\pycaret_env\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
C:\Users\andre\AppData\Local\Temp\ipykernel_7904\4019339277.py:18: UserWarning: PySoundF

Características de audio (beat-sync) guardadas en: audio_features\audio_features_beat.csv
                                             song_id  \
0                                2#40 - Samad Savage   
1  21 Seconds - DJ Swiss & Dan Da Man Remix Vocal...   
2         Absolutist Regeneration - Deathspell Omega   
3                      Act Out - Leroy George Wilkes   
4  Ain't No Way [Feat. 6LACK, Rico Nasty, J.I.D, ...   

                                       beat_features  
0  [[0.1179114505648613, 0.11783639341592789, 0.2...  
1  [[0.6232354044914246, 0.7310153841972351, 0.68...  
2  [[0.6578890085220337, 0.5009267330169678, 0.56...  
3  [[0.19272536039352417, 0.16313757002353668, 0....  
4  [[0.2387864887714386, 0.08314859867095947, 0.1...  


In [ ]:
# --- Paso 4: Combinar con DataFrame Principal (opcional)
df_combined = pd.merge(df_lyric, df_audio_features, on='songId', how='left')
print(df_combined.head())

In [ ]:
"""
# --- Main Script ---
audio_dir = 'Canciones/'
audio_file_paths = []
audio_features = []

# Get list of valid .wav files
for filename in os.listdir(audio_dir):
    if filename.lower().endswith(".wav"):
        full_path = os.path.join(audio_dir, filename)
        audio_file_paths.append(full_path)
        print(f"Found file: {full_path}")

# Process files
for audio_file in audio_file_paths:
    try:
        # Load with librosa, specifying dtype for 32-bit float compatibility
        y, sr = librosa.load(audio_file, sr=None, dtype=np.float32)

        # Feature extraction
        y_harmonic, y_percussive = librosa.effects.hpss(y)
        tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=512, n_mfcc=13)
        mfcc_delta = librosa.feature.delta(mfcc)
        beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)
        chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)
        beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)
        beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

        audio_features.append(beat_features)
        print(f"Successfully processed: {audio_file}")

    except Exception as e:
        print(f"Failed to process {audio_file}: {str(e)}")

# Save features (handle empty list case)
if audio_features:  # Check if the list is not empty
  np.save('audio_features.npy', audio_features)
  print("Feature extraction completed successfully!")
else:
  print("No features were extracted.")
"""

Found file: Canciones/2#40 - Samad Savage.wav
Found file: Canciones/21 Seconds - DJ Swiss & Dan Da Man Remix Vocal - So Solid Crew.wav
Found file: Canciones/Absolutist Regeneration - Deathspell Omega.wav
Found file: Canciones/Act Out - Leroy George Wilkes.wav
Found file: Canciones/Ain't No Way [Feat. 6LACK, Rico Nasty, J.I.D, Jasiah, Kitty Ca$h] - Denzel Curry.wav
Found file: Canciones/Alice's Restaurant Massacree - Arlo Guthrie.wav
Found file: Canciones/Angels Hung from the Arches of Heaven - Goatwhore.wav
Found file: Canciones/Arise Again - Soulfly.wav
Found file: Canciones/Bad Bitchez - Paul Wall.wav
Found file: Canciones/Bamboo Banga - DJ Eli Remix - M.I.A..wav
Found file: Canciones/Beno! - Playboi Carti.wav
Found file: Canciones/Benz - Samsonyte.wav
Found file: Canciones/Better Dead Than Slave - Violentor.wav
Found file: Canciones/Blunt After Blunt - Danny Brown.wav
Found file: Canciones/Caesura - Neaera.wav
Found file: Canciones/Casanova (feat. SSIO) - Farid Bang.wav
Found file: 

C:\Users\andre\AppData\Local\Temp\ipykernel_22072\2941872878.py:17: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=None, dtype=np.float32)
c:\Users\andre\anaconda3\envs\pycaret_env\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Successfully processed: Canciones/2#40 - Samad Savage.wav


C:\Users\andre\AppData\Local\Temp\ipykernel_22072\2941872878.py:17: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=None, dtype=np.float32)
c:\Users\andre\anaconda3\envs\pycaret_env\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Successfully processed: Canciones/21 Seconds - DJ Swiss & Dan Da Man Remix Vocal - So Solid Crew.wav
Successfully processed: Canciones/Absolutist Regeneration - Deathspell Omega.wav
Successfully processed: Canciones/Act Out - Leroy George Wilkes.wav
Successfully processed: Canciones/Ain't No Way [Feat. 6LACK, Rico Nasty, J.I.D, Jasiah, Kitty Ca$h] - Denzel Curry.wav
